#Load and prepare data

In [15]:
#libraries used
import open3d as o3d
import numpy as np

In [17]:
#create paths and load data
input_path="/home/jm/workspace/Open3D-master/examples/test_data/"
output_path="/home/jm/workspace/Open3D-master/examples/test_data/result"
dataname="acc_w_normal.pcd"
#pcd 파일 사용시 11 row 까지 skip
point_cloud= np.loadtxt(input_path+dataname,skiprows=11) 

In [18]:
#Format to open3d usable objects
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
# pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,3:6])

#Choose a meshing startegy

In [19]:
#radius determination
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [20]:
#compute bpa mesh
bpa_mesh=o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius,radius*2]))

In [21]:
#decimating the mesh
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)
# dec_mesh.remove_degenerate_triangles()
# dec_mesh.remove_duplicated_triangles()
# dec_mesh.remove_duplicated_vertices()
# dec_mesh.remove_non_manifold_edges()

In [22]:
#compute poisson mesh
poisson_mesh=o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd,depth=8,width=0,scale=1.1,linear_fit=False)[0]

In [23]:
#cropping
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

#Export and visualize

In [24]:
#export
o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh)
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop)

True

In [25]:
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [26]:
##execution of function
my_lods = lod_mesh_export(bpa_mesh, [100000,50000,10000,1000,100], ".ply", output_path)

generation of 100 LoD successful


In [27]:
my_lods2 = lod_mesh_export(bpa_mesh, [8000,800,300], ".ply", output_path)

generation of 300 LoD successful


In [28]:
#visualize
# o3d.visualization.draw_geometries([my_lods[100]])
# o3d.visualization.draw_geometries([my_lods[1000]])
# o3d.visualization.draw_geometries([my_lods[10000]])
# o3d.visualization.draw_geometries([my_lods[50000]])
o3d.visualization.draw_geometries([my_lods[100000]])

In [29]:
# o3d.visualization.draw_geometries([my_lods2[300]])
# o3d.visualization.draw_geometries([my_lods2[800]])
o3d.visualization.draw_geometries([my_lods2[8000]])